## Benchmark model

In this notebook a Simple Moving Average modele will be created in order to have a benchmark for my DeepAR based model (to be tuned in the next notebook).

### Hyperparameters

DeepAR is the model of choice of this project.
This model expects input data to be already test-train splitted.
A big part of the model design has to be done looking close at data.
More specifically, defining these two hyperparameters about the data:
* Context length
* Prediction length

### Prediction length

This is the length of the time series future predictions in days. It will be conveniently set to 5 days (exactly a week of trading hours) because a shorter interval would be of little significance.
A longer interval could be interesting from an application point of view, but it can be challenging in terms of model performances.

### Context length

Context length can be either:
* designed on patterns or seasonality observed in the data, if any is present;
* chosen as a fixed value. This will be my choice, and it will be the same as the moving average window, in order to have a good reference metrics, applicable to both this model and the benchmark model.

To explore this second option, we will refer to what we've found during the EDA stage.

In [98]:
prediction_length = [10, 20, 50]
context_length = [10, 20, 50]

### Train, test and validation split

Time series will be all trimmed from `data_start` in `1.ExploratoryDataAnalysis.ipy`, according to DeepAR documentation, train time series should be set as the size of the entire time series less the prediction length.
Validation length will exactly equal to prediction length.

In [99]:
from data.data_utils import train_test_valid_split

#### IBM Stock train, test and validation split

In [100]:
df_ibm = df_ibm.loc[data_start:].copy()

In [101]:
df_ibm_train, df_ibm_test, df_ibm_valid = train_test_valid_split(df_ibm, prediction_length=prediction_length[1])
print(len(df_ibm), len(df_ibm_train), len(df_ibm_test), len(df_ibm_valid))

4165 4125 4145 20


In [102]:
df_ibm_train.head()

,Open,High,Low,Close,Adj Close,Volume,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,
2004-08-19,84.750000,85.349998,84.449997,84.889999,54.979382,4704500,54.397126,54.958798,55.913001,55.525647,53.268605,56.500281,53.417314,59.046566,52.779437
2004-08-20,84.519997,85.250000,84.519997,85.250000,55.212502,4501400,54.511759,54.977556,55.852774,55.720874,53.302644,56.521944,53.433168,58.920109,52.785438
2004-08-23,85.230003,85.449997,84.650002,84.650002,54.823910,4260600,54.582999,54.969128,55.779992,55.771068,53.394930,56.515013,53.423244,58.766151,52.793832
2004-08-24,85.000000,85.150002,84.349998,84.709999,54.862782,2710400,54.564866,54.937116,55.713028,55.726906,53.402826,56.462835,53.411398,58.625949,52.800108
2004-08-25,85.000000,85.269997,84.550003,85.070000,55.095955,4405600,54.654243,54.917732,55.644653,55.829713,53.478772,56.423940,53.411524,58.447571,52.841736


In [103]:
df_ibm_test.head()

,Open,High,Low,Close,Adj Close,Volume,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,
2004-08-19,84.750000,85.349998,84.449997,84.889999,54.979382,4704500,54.397126,54.958798,55.913001,55.525647,53.268605,56.500281,53.417314,59.046566,52.779437
2004-08-20,84.519997,85.250000,84.519997,85.250000,55.212502,4501400,54.511759,54.977556,55.852774,55.720874,53.302644,56.521944,53.433168,58.920109,52.785438
2004-08-23,85.230003,85.449997,84.650002,84.650002,54.823910,4260600,54.582999,54.969128,55.779992,55.771068,53.394930,56.515013,53.423244,58.766151,52.793832
2004-08-24,85.000000,85.150002,84.349998,84.709999,54.862782,2710400,54.564866,54.937116,55.713028,55.726906,53.402826,56.462835,53.411398,58.625949,52.800108
2004-08-25,85.000000,85.269997,84.550003,85.070000,55.095955,4405600,54.654243,54.917732,55.644653,55.829713,53.478772,56.423940,53.411524,58.447571,52.841736


In [104]:
df_ibm_valid.head()

,Open,High,Low,Close,Adj Close,Volume,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,
2021-02-05,121.000000,121.809998,120.519997,121.790001,120.183998,4563600,118.875485,122.591825,123.061745,121.711760,116.039211,131.798370,113.385281,129.295405,116.828085
2021-02-08,122.620003,123.980003,122.400002,123.610001,121.980003,5888000,119.371852,122.349069,123.045759,122.485821,116.257883,131.337979,113.360160,129.286471,116.805047
2021-02-09,121.900002,122.599998,121.050003,122.099998,122.099998,4717000,119.494376,122.109846,123.036514,122.948953,116.039799,130.841518,113.378173,129.281518,116.791511
2021-02-10,123.029999,123.410004,121.209999,122.239998,122.239998,4866800,119.632872,121.846537,123.027110,123.424498,115.841246,130.202563,113.490512,129.275534,116.778685
2021-02-11,122.000000,122.209999,120.629997,120.910004,120.910004,5380600,119.874217,121.629720,123.007486,123.651596,116.096838,129.838037,113.421403,129.276600,116.738372


#### Apple Inc. Stock train test split

In [105]:
df_aapl = df_aapl.loc[data_start:].copy()

In [106]:
df_aapl_train, df_aapl_test, df_aapl_valid = train_test_valid_split(df_aapl, prediction_length=prediction_length[1])
print(len(df_aapl), len(df_aapl_train), len(df_aapl_test), len(df_aapl_valid))

4165 4125 4145 20


In [107]:
df_aapl_train.head()

,Open,High,Low,Close,Adj Close,Volume,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,
2004-08-19,0.562679,0.568929,0.542143,0.548393,0.472366,388920000,0.473627,0.481141,0.484422,0.491197,0.456057,0.504857,0.457425,0.519835,0.449008
2004-08-20,0.548393,0.553393,0.544464,0.550000,0.473750,316780800,0.475196,0.481218,0.484606,0.488983,0.461409,0.504821,0.457614,0.519691,0.449521
2004-08-23,0.551071,0.558393,0.546429,0.555000,0.478057,254660000,0.476395,0.481079,0.484711,0.488652,0.464139,0.504725,0.457433,0.519683,0.449738
2004-08-24,0.558214,0.570536,0.556964,0.570536,0.491439,374136000,0.477057,0.480710,0.485274,0.491797,0.462317,0.503401,0.458019,0.519742,0.450806
2004-08-25,0.569107,0.591964,0.566607,0.590179,0.508359,505618400,0.480195,0.481310,0.486000,0.504872,0.455517,0.506265,0.456355,0.520858,0.451141


In [108]:
df_aapl_test.head()

,Open,High,Low,Close,Adj Close,Volume,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,
2004-08-19,0.562679,0.568929,0.542143,0.548393,0.472366,388920000,0.473627,0.481141,0.484422,0.491197,0.456057,0.504857,0.457425,0.519835,0.449008
2004-08-20,0.548393,0.553393,0.544464,0.550000,0.473750,316780800,0.475196,0.481218,0.484606,0.488983,0.461409,0.504821,0.457614,0.519691,0.449521
2004-08-23,0.551071,0.558393,0.546429,0.555000,0.478057,254660000,0.476395,0.481079,0.484711,0.488652,0.464139,0.504725,0.457433,0.519683,0.449738
2004-08-24,0.558214,0.570536,0.556964,0.570536,0.491439,374136000,0.477057,0.480710,0.485274,0.491797,0.462317,0.503401,0.458019,0.519742,0.450806
2004-08-25,0.569107,0.591964,0.566607,0.590179,0.508359,505618400,0.480195,0.481310,0.486000,0.504872,0.455517,0.506265,0.456355,0.520858,0.451141


In [109]:
df_aapl_valid.head()

,Open,High,Low,Close,Adj Close,Volume,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,
2021-02-05,137.350006,137.419998,135.860001,136.759995,136.759995,75524000,137.256334,134.158744,129.253941,145.233649,129.279019,144.148255,124.169232,142.671036,115.836847
2021-02-08,136.029999,136.960007,134.919998,136.910004,136.910004,71297200,136.676659,134.411596,129.692178,143.676250,129.677068,144.411367,124.411824,142.631632,116.752725
2021-02-09,136.619995,137.880005,135.850006,136.009995,136.009995,76774200,135.983018,134.772718,130.095241,141.421996,130.544040,144.433219,125.112217,142.520978,117.669504
2021-02-10,136.479996,136.990005,134.399994,135.389999,135.389999,73046600,135.337215,135.111827,130.474720,138.886435,131.787994,144.327034,125.896619,142.341671,118.607770
2021-02-11,135.899994,136.389999,133.770004,135.130005,135.130005,64154400,135.161670,135.333592,130.799873,138.540093,131.783247,144.311647,126.355537,142.252958,119.346788


#### Amazon Stock train test split

In [110]:
df_amzn = df_amzn.loc[data_start:].copy()

In [111]:
df_amzn_train, df_amzn_test, df_amzn_valid = train_test_valid_split(df_amzn, prediction_length=prediction_length[1])
print(len(df_amzn), len(df_amzn_train), len(df_amzn_test), len(df_amzn_valid))

4165 4125 4145 20


In [112]:
df_amzn_train.head()

,Open,High,Low,Close,Adj Close,Volume,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,
2004-08-19,40.259998,40.480000,36.349998,38.630001,38.630001,12696100,37.106001,37.6520,44.9828,39.862539,34.349462,40.417963,34.886038,57.693587,32.272014
2004-08-20,38.459999,39.910000,38.110001,39.509998,39.509998,6790800,37.508000,37.6285,44.7682,40.386916,34.629084,40.318115,34.938885,57.479021,32.057379
2004-08-23,39.889999,40.000000,39.110001,39.450001,39.450001,5532600,37.921000,37.6620,44.5584,40.581550,35.260450,40.427620,34.896380,57.267085,31.849715
2004-08-24,39.720001,39.930000,38.320000,39.049999,39.049999,7640400,38.116000,37.6590,44.3544,40.794891,35.437110,40.418129,34.899871,57.083144,31.625656
2004-08-25,39.060001,40.490002,38.160000,40.299999,40.299999,7254800,38.490000,37.7755,44.1582,41.246567,35.733433,40.776113,34.774887,56.827117,31.489283


In [113]:
df_amzn_test.head()

,Open,High,Low,Close,Adj Close,Volume,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,
2004-08-19,40.259998,40.480000,36.349998,38.630001,38.630001,12696100,37.106001,37.6520,44.9828,39.862539,34.349462,40.417963,34.886038,57.693587,32.272014
2004-08-20,38.459999,39.910000,38.110001,39.509998,39.509998,6790800,37.508000,37.6285,44.7682,40.386916,34.629084,40.318115,34.938885,57.479021,32.057379
2004-08-23,39.889999,40.000000,39.110001,39.450001,39.450001,5532600,37.921000,37.6620,44.5584,40.581550,35.260450,40.427620,34.896380,57.267085,31.849715
2004-08-24,39.720001,39.930000,38.320000,39.049999,39.049999,7640400,38.116000,37.6590,44.3544,40.794891,35.437110,40.418129,34.899871,57.083144,31.625656
2004-08-25,39.060001,40.490002,38.160000,40.299999,40.299999,7254800,38.490000,37.7755,44.1582,41.246567,35.733433,40.776113,34.774887,56.827117,31.489283


In [114]:
df_amzn_valid.head()

,Open,High,Low,Close,Adj Close,Volume,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,
2021-02-05,3319.00000,3377.000000,3302.709961,3352.149902,3352.149902,3613600,3301.508984,3240.689978,3210.729790,3416.993362,3186.024606,3424.052246,3057.327711,3359.347489,3062.112091
2021-02-08,3358.50000,3365.000000,3304.000000,3322.939941,3322.939941,3257400,3304.402979,3247.701978,3214.827388,3420.499859,3188.306098,3432.447621,3062.956334,3364.311500,3065.343275
2021-02-09,3312.48999,3338.000000,3297.840088,3305.000000,3305.000000,2203500,3302.289990,3257.241479,3217.225986,3417.394278,3187.185702,3432.419542,3082.063417,3368.598027,3065.853945
2021-02-10,3314.00000,3317.949951,3254.000000,3286.580078,3286.580078,3151600,3307.689990,3265.528979,3219.050786,3412.900832,3202.479148,3428.813657,3102.244302,3371.541619,3066.559953
2021-02-11,3292.00000,3292.000000,3248.060059,3262.129883,3262.129883,2295800,3310.140967,3270.340979,3220.932583,3409.050210,3211.231724,3426.791380,3113.890578,3373.176069,3068.689097


#### Alphabet Inc. Stock train test split

In [115]:
df_googl = df_googl.loc[data_start:].copy()

In [116]:
df_googl_train, df_googl_test, df_googl_valid = train_test_valid_split(df_googl, prediction_length=prediction_length[1])
print(len(df_googl), len(df_googl_train), len(df_googl_test), len(df_googl_valid))

4165 4125 4145 20


In [117]:
df_googl_train.head()

,Open,High,Low,Close,Adj Close,Volume,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,
2004-08-19,50.050049,52.082081,48.028027,50.220219,50.220219,44659096,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-20,50.555557,54.594597,50.300301,54.209209,54.209209,22834343,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-23,55.430431,56.796799,54.579578,54.754753,54.754753,18256126,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-24,55.675674,55.855858,51.836838,52.487488,52.487488,15247337,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-25,52.532532,54.054054,51.991993,53.053055,53.053055,9188602,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381


In [118]:
df_googl_test.head()

,Open,High,Low,Close,Adj Close,Volume,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,
2004-08-19,50.050049,52.082081,48.028027,50.220219,50.220219,44659096,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-20,50.555557,54.594597,50.300301,54.209209,54.209209,22834343,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-23,55.430431,56.796799,54.579578,54.754753,54.754753,18256126,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-24,55.675674,55.855858,51.836838,52.487488,52.487488,15247337,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-25,52.532532,54.054054,51.991993,53.053055,53.053055,9188602,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381


In [119]:
df_googl_valid.head()

,Open,High,Low,Close,Adj Close,Volume,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,
2021-02-05,2059.560059,2095.939941,2050.000000,2088.830078,2088.830078,1489700,1931.525964,1862.692487,1803.526794,2130.534575,1732.517353,2080.053938,1645.331036,1977.481085,1629.572504
2021-02-08,2100.000000,2115.000000,2063.350098,2084.520020,2084.520020,1441400,1950.549963,1877.026990,1809.939194,2169.142184,1731.957743,2113.362723,1640.691257,2000.752092,1619.126296
2021-02-09,2074.800049,2098.510010,2070.000000,2075.389893,2075.389893,1094600,1967.293958,1892.981982,1816.164392,2196.764256,1737.823659,2137.925990,1648.037975,2020.693732,1611.635052
2021-02-10,2083.139893,2098.879883,2052.239990,2086.479980,2086.479980,1255100,1994.047961,1910.434479,1822.153591,2208.540662,1779.555260,2158.433512,1662.435445,2040.285300,1604.021882
2021-02-11,2091.000000,2094.000000,2068.860107,2088.750000,2088.750000,1021200,2017.602966,1927.509479,1828.840591,2214.352044,1820.853889,2175.226576,1679.792381,2058.679926,1599.001255


### Metrics computation

Computing metrics on the benchmark model will give me a good reference to evaluate the deep learning model after training.

As can be presumed and seen from data prints, first `n` values of validation data are `NaN` so, it will be better just to exclude the first `n-1` values from the validation set to avoid misleading values to be catch in metrics evaluation. 

In [120]:
from sklearn.metrics import mean_squared_error

#### IBM stock

In [121]:
#n = 10
#ma_str = str(n)+'_ac_ma'

In [122]:
#ibm_ma_mse_loss = mean_squared_error(df_ibm_valid.iloc[n-1:]['Adj Close'], df_ibm_valid.iloc[n-1:][ma_str])

In [123]:
#print(ibm_ma_mse_loss)

In [124]:
n = 20
ma_str = str(n)+'_ac_ma'

In [125]:
ibm_ma_mse_loss = mean_squared_error(df_ibm_valid.iloc[n-1:]['Adj Close'], df_ibm_valid[n-1:][ma_str])

In [126]:
print(ibm_ma_mse_loss)

3.2536992952007755


In [127]:
#n = 50
#ma_str = str(n)+'_ac_ma'

In [128]:
#ibm_ma_mse_loss = mean_squared_error(df_ibm_valid.iloc[n-1:]['Adj Close'], df_ibm_valid[n-1:][ma_str])

In [129]:
#print(ibm_ma_mse_loss)

#### Apple Inc. stock

In [130]:
#n = 10
#ma_str = str(n)+'_ac_ma'

In [131]:
#aapl_ma_mse_loss = mean_squared_error(df_aapl_valid.iloc[n-1:]['Adj Close'], df_aapl_valid.iloc[n-1:][ma_str])

In [132]:
#print(aapl_ma_mse_loss)

In [133]:
n = 20
ma_str = str(n)+'_ac_ma'

In [134]:
aapl_ma_mse_loss = mean_squared_error(df_aapl_valid.iloc[n-1:]['Adj Close'], df_aapl_valid[n-1:][ma_str])

In [135]:
print(aapl_ma_mse_loss)

53.846262362916605


In [136]:
#n = 50
#ma_str = str(n)+'_ac_ma'

In [137]:
#aapl_ma_mse_loss = mean_squared_error(df_aapl_valid.iloc[n-1:]['Adj Close'], df_aapl_valid[n-1:][ma_str])

In [138]:
#print(aapl_ma_mse_loss)

#### Amazon.com stock

In [139]:
#n = 10
#ma_str = str(n)+'_ac_ma'

In [140]:
#amzn_ma_mse_loss = mean_squared_error(df_amzn_valid.iloc[n-1:]['Adj Close'], df_amzn_valid.iloc[n-1:][ma_str])

In [141]:
#print(amzn_ma_mse_loss)

In [142]:
n = 20
ma_str = str(n)+'_ac_ma'

In [143]:
amzn_ma_mse_loss = mean_squared_error(df_amzn_valid.iloc[n-1:]['Adj Close'], df_amzn_valid[n-1:][ma_str])

In [144]:
print(amzn_ma_mse_loss)

37279.69501723785


In [145]:
#n = 50
#ma_str = str(n)+'_ac_ma'

In [146]:
#amzn_ma_mse_loss = mean_squared_error(df_amzn_valid.iloc[n-1:]['Adj Close'], df_amzn_valid[n-1:][ma_str])

In [147]:
#print(amzn_ma_mse_loss)

#### Alphabet Inc. stock

In [148]:
#n = 10
#ma_str = str(n)+'_ac_ma'

In [149]:
#googl_ma_mse_loss = mean_squared_error(df_googl_valid.iloc[n-1:]['Adj Close'], df_googl_valid.iloc[n-1:][ma_str])

In [150]:
#print(googl_ma_mse_loss)

In [151]:
n = 20
ma_str = str(n)+'_ac_ma'

In [152]:
googl_ma_mse_loss = mean_squared_error(df_googl_valid.iloc[n-1:]['Adj Close'], df_googl_valid[n-1:][ma_str])

In [153]:
print(googl_ma_mse_loss)

590.1521857801364


In [154]:
#n = 50
#ma_str = str(n)+'_ac_ma'

In [155]:
#googl_ma_mse_loss = mean_squared_error(df_googl_valid.iloc[n-1:]['Adj Close'], df_googl_valid[n-1:][ma_str])

In [156]:
#print(googl_ma_mse_loss)

Volatility

In [157]:
print(volatility(df_ibm_valid['Adj Close'], n))

0.07470456637836545


In [158]:
print(volatility(df_aapl_valid['Adj Close'], n))

1.760548166395322


In [159]:
print(volatility(df_amzn_valid['Adj Close'], n))

727.8584384232057


In [160]:
print(volatility(df_googl_valid['Adj Close'], n))

48.983635914739715


As expected, loss augments as we observe moving average on larger windows.
Also, we can observe that Amazon.com and Alphabet Inc. have greater losses, that also corresponds to higher volatility.